# ROPE Demo on Google Colab

Run the ROPE demo on a Colab GPU and download the results.

**Before starting:**
1. **Runtime → Change runtime type → GPU** (T4 or A100).
2. Get a [HuggingFace token](https://huggingface.co/settings/tokens) if you want to use Llama (phi2 works without it).

**Option A:** You have the ROPE repo on GitHub → set `REPO_URL` below and run the clone cell.

**Option B:** No GitHub → zip your local ROPE folder, upload it in cell 2, then run the rest.

## 1. Install dependencies and clone or use uploaded ROPE

In [1]:
import os

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU. Set Runtime → Change runtime type → GPU.")

# Option A: Clone from GitHub (set your repo URL)
REPO_URL = "https://github.com/zacharias1219/ROPE.git"  # Change to your repo, or leave and use Option B
REPO_DIR = "ROPE"

if not os.path.isdir(REPO_DIR):
    if "yourusername" in REPO_URL:
        print("Set REPO_URL to your repo, or upload a zip of the ROPE folder (Option B below).")
    else:
        !git clone --depth 1 "{REPO_URL}" "{REPO_DIR}"

if os.path.isdir(REPO_DIR):
    %cd {REPO_DIR}
    !pip install -e . -q
    print("ROPE installed.")
else:
    print("ROPE folder not found. Upload a zip in the next cell (Option B).")

GPU: NVIDIA A100-SXM4-40GB
/content/ROPE
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rope-bench (pyproject.toml) ... done
ROPE installed.


## 2. Option B: Upload ROPE zip (if you didn't clone)

Zip your local `ROPE` project folder (include `rope/`, `data/`, `pyproject.toml`, etc.), upload below, then re-run cell 1 logic in the next cell.

In [ ]:
# Run this only if you're uploading a zip. Upload the zip when prompted.
from google.colab import files
import zipfile

uploaded = files.upload()  # Opens file picker
for name in uploaded:
    if name.endswith('.zip'):
        with zipfile.ZipFile(name, 'r') as z:
            z.extractall('.')
        print(f"Extracted {name}")
        break
# If zip had a single root folder (e.g. ROPE/), we're in that folder now or in content.
# Go to the folder that contains pyproject.toml
import os
for d in ['ROPE', '.', 'rope-bench']:
    if os.path.isfile(os.path.join(d, 'pyproject.toml')):
        %cd {d}
        !pip install -e . -q
        print(f"Installed from {d}")
        break

## 3. HuggingFace login (for Llama models)

In [2]:
from getpass import getpass
from huggingface_hub import login

token = getpass("Paste your HuggingFace token (or press Enter to skip; phi2 works without it): ")
if token.strip():
    login(token=token.strip())
    print("Login successful. Gated models (Llama) will now work.")
else:
    print("Skipped. Use phi2-only runs (e.g. rope run --models phi2) or add token for Llama.")

Login successful. Gated models (Llama) will now work.


## 4. Run ROPE demo

In [3]:
# Standard demo: llama2-7b + 2 defenses + 20 attacks (~5–15 min on T4)
!rope demo

Running ROPE demo...

Loading tasks and attacks...
  Loaded 30 tasks, 20 attacks

Loading judge model (llama3-8b)...
Loading llama3-8b (meta-llama/Meta-Llama-3-8B-Instruct)...
config.json: 100% 654/654 [00:00<00:00, 2.88MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
model.safetensors.index.json: 100% 23.9k/23.9k [00:00<00:00, 5.17MB/s]
Fetching 4 files: 100% 4/4 [00:53<00:00, 13.44s/it]
Download complete: 100% 16.1G/16.1G [00:53<00:00, 298MB/s]                
Loading weights: 100% 291/291 [00:05<00:00, 56.35it/s, Materializing param=model.norm.weight]                              
generation_config.json: 100% 187/187 [00:00<00:00, 1.04MB/s]
tokenizer_config.json: 100% 51.0k/51.0k [00:00<00:00, 4.77MB/s]
tokenizer.json: 100% 9.09M/9.09M [00:00<00:00, 11.7MB/s]
special_tokens_map.json: 100% 73.0/73.0 [00:00<00:00, 435kB/s]
  Loaded llama3-8b on cuda:0

EVALUATING: llama2-7b
Loading llama2-7b (meta-llama/Llama-2-7b-chat-hf)...
config.json: 100% 614/614 [00:00<00:00, 3.28MB/s]
mo

## 5. View results

In [4]:
import json
import pandas as pd
import os

with open("demo_results.json") as f:
    results = json.load(f)

# Try to load CSV, or compute from JSON if missing
if os.path.exists("demo_results_metrics.csv"):
    metrics = pd.read_csv("demo_results_metrics.csv")
else:
    # Compute metrics from JSON (for older rope demo runs)
    from rope.metrics import compute_metrics
    metrics = compute_metrics(results)
    print("Note: CSV not found, computed metrics from JSON.")

print("Summary metrics:")
display(metrics)
print(f"\nTotal evaluations: {len(results)}")
print("\nSample result:")
print(json.dumps(results[0], indent=2))

FileNotFoundError: [Errno 2] No such file or directory: 'demo_results_metrics.csv'

## 6. Download result files

In [ ]:
from google.colab import files

for name in ["demo_results.json", "demo_results_metrics.csv", "demo_results_report.txt"]:
    if os.path.exists(name):
        files.download(name)
        print(f"Downloaded {name}")
    else:
        print(f"Not found: {name}")